In [ ]:
import pandas as pd
import re
import glob


In [ ]:
folder_path = 'I:/Expense tracking/Apple Pay'
# folder_path='D:/Personal Projects/Expense tracking/Apple Pay'
file_pattern = folder_path + '/*.csv'  # Adjust the pattern if using other formats, e.g., '*.xlsx'

# Use glob to find all files matching the pattern
file_list = glob.glob(file_pattern)

# Read all files and concatenate them into a single DataFrame
df_apple_pay = pd.concat((pd.read_csv(file) for file in file_list), ignore_index=True)

In [ ]:
df_apple_pay.head()

In [ ]:
# df_apple_pay['Transaction Date'] = pd.to_datetime(df_apple_pay['Transaction Date'])
# df_apple_pay['Transaction Date']=df_apple_pay['Transaction Date'].dt.strftime('%m/%d/%y')
# # df_apple_pay['Transaction Date']=pd.to_datetime(df_apple_pay['Transaction Date'], format='%m/%d/%y')

In [ ]:
df_apple_pay=df_apple_pay[['Transaction Date','Description','Category','Amount (USD)','Merchant']]
df_apple_pay['Institution']='Apple Pay Credit Card'
df_apple_pay['Is Hidden']='No'
df_apple_pay['Is Pending']='No'
df_apple_pay['Account']='APPLE PAY'


In [ ]:
df_apple_pay.rename(columns={"Merchant":"Standardized Description","Amount (USD)":"Amount"},inplace=True)
dtype_dict = {
    'Transaction Date':'datetime64[ns]',
    'Description': 'string',
    'Institution': 'string',
    'Account': 'string',
    'Category': 'string',
    'Is Hidden': 'string',
    'Is Pending': 'string',
    'Amount': 'float64',
    'Standardized Description': 'string'
}
df_apple_pay = df_apple_pay.astype(dtype_dict)
# df_apple_pay['Transaction Date']=df_apple_pay['Transaction Date'].dt.strftime('%m/%d/%y')
df_apple_pay=df_apple_pay[['Transaction Date','Description','Institution','Account','Category','Is Hidden','Is Pending','Amount','Standardized Description']]


In [ ]:
df_apple_pay.head()

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [ ]:
def convert_accounting_to_decimal(amount):
    # Remove parentheses and commas
    amount = amount.replace('$', '').replace('(', '-').replace(')', '').replace(',', '')
    # Convert to float
    return float(amount)

In [ ]:

file_names = ['Transactions 2022_06-2023_06.csv', 'Transactions 2023_06-2024_06.csv']
folder_path = 'I:/Expense tracking'
# folder_path='D:/Personal Projects/Expense tracking'
file_pattern = folder_path + '/*.csv'  # Adjust the pattern if using other formats, e.g., '*.xlsx'

# Use glob to find all files matching the pattern
file_list = glob.glob(file_pattern)
print(file_list)

# Load and concatenate the files into a single DataFrame
df = pd.concat([pd.read_csv(file) for file in file_names], ignore_index=True)


In [ ]:
df['Amount']=df['Amount'].apply(convert_accounting_to_decimal)

In [ ]:
df['Standardized Description']=''
df.rename(columns={"Date":"Transaction Date"},inplace=True)


In [ ]:

dtype_dict = {
    'Transaction Date':'datetime64[ns]',
    'Description': 'string',
    'Institution': 'string',
    'Account': 'string',
    'Category': 'string',
    'Is Hidden': 'string',
    'Is Pending': 'string',
    'Amount': 'float64',
    'Standardized Description': 'string'
}
df = df.astype(dtype_dict)
# len(df)

In [ ]:
df['Account'].unique()

In [ ]:
len(df)

In [ ]:
df_full=pd.concat([df,df_apple_pay],ignore_index=True)

In [ ]:
len(df_full)

In [ ]:
print(df_full.head()

In [ ]:
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')
df_full[df_full['Account']=='APPLE PAY']['Category'].unique()
df_full[df_full['Category']=='Medical']

In [ ]:
df_full.head()

In [ ]:
df_full['Category'].unique()

In [ ]:
patterns_map = {
    'COFFEE': [r'STARBUCKS', r'DUNKIN',r'PANERA'],
    'SHOPPING':[r'WALMART',r'TARGET'],
    'RENT':[r'BELL WEB',r'SUMMIT',r'PALMS ASSOCIATES'],
    'ENERGY, GAS & ELECTRIC':[r'TXU'],
    'RESTAURANTS/DINING':[r'HELLO INDIA',r'HELLO! INDIA'],
    'SUBSCRIPTIONS':[r'ENVATO',r'STRATA',r'VIDIQ',r'YOUTUBE',r'HUNTER.IO'],
    'PAYMENT':[r'USCIS',r'ZELLE PAYMENT TO'],
    'INCOME':[r'ZELLE PAYMENT FROM'],
    'ENTERTAINMENT':[r'SIXFLAGS'],
    'GYM':[r'CRUNCH']
}

# Function to map the new category
def map_category(row):
    description_upper = row['Description'].upper()  # Convert to uppercase for case-insensitive matching
    for category, patterns in patterns_map.items():
        if any(re.search(pattern, description_upper) for pattern in patterns):
            return category
    return row['Category'].upper()
    
df_full['Category'] = df_full.apply(map_category, axis=1)



In [ ]:
master_category_dict={
'SHOPPING/GROCERIES':['Groceries','Grocery','Shopping','Furniture & Home Decor','Merchandise/Misc','Gifts','Clothing','Home Improvement/Maintenance','Books','Electronics & Software'],
'ENTERTAINMENT':['Entertainment','Air Travel','Hotel','Rental Car','Travel & Vacation'],
'FOOD AND DINING':['Restaurants','Restaurants/Dining','Fast Food & Convenience',	'Coffee','Food','Alcohol & Bars'],
'SUBSCRIPTIONS':['Subscriptions','Education'],
'HEALTH':['Gym'],
'BILLS AND UTILITIES':['Phone, Internet & Cable','Gas & Fuel','Gas','Insurance','Auto Service','Energy, Gas & Electric','Auto & Transport',
                       	'Water'	,'Bills & Utilities','Rent','Tolls','Rideshare','Parking','Pharmacy','Hair & Nails','Medical','Household Services'],
'MISCELLANEOUS':['Transfers','Credit Card Payment','Savings','Investment Savings','Investment Transfers','Online Payment','Bank Fee'
                 	,'Fees & Charges','Shipping & Handling','Taxes','Cash/ATM','Charity','Payment','Other'],
'INCOME':['Interest Income','Paycheck/Salary','Tax Refund','Income']
    
}

def map_category(row):
    for category, patterns in master_category_dict.items():
        if any(pattern.lower() in row['Category'].lower() for pattern in patterns):
            return category
    return 'OTHER'
    
df_full['Master Category'] = df_full.apply(map_category, axis=1)


In [ ]:
df_full['Master Category'].unique()

In [ ]:
df_full.head()

In [ ]:
desc_map = {
    'STARBUCKS': [r'STARBUCKS'],
    'DUNKIN': [r'DUNKIN'],
    '7-ELEVEN':[r'7-ELEVEN'],
    'WALMART':[r'WALMART',r'WM',r'WAL-MART'],
    'TARGET':[r'TARGET'],
    'HOME DEPOT':[r'HOME DEPOT'],
    'AMAZON':[r'AMAZON',r'AMZN'],
    'COSTCO':[r'COSTCO'],
    'HELLO INDIA': [r'HELLO INDIA',r'HELLO! INDIA']
}

def map_description(row):
    description_upper = row['Description'].upper()  # Convert to uppercase for case-insensitive matching
    for category, patterns in desc_map.items():
        if any(re.search(pattern, description_upper) for pattern in patterns):
            return category
    return row['Description'].upper()
    

# Apply the function to create a new column 'Mapped Description'
df_full['Standardized Description'] = df_full.apply(map_description,axis=1)


Filter out 7 ELEVEN as it needs cleaning

In [ ]:

filtered_df = df_full[~(df_full['Description'].str.match('7-ELEVEN', case=False, na=False) & (df_full['Category'] == 'FAST FOOD & CONVENIENCE'))]


7-ELEVEN 

In [ ]:
df_7_eleven = df_full[df_full['Description'].str.match('7-ELEVEN', case=False, na=False) & (df_full['Category']=='FAST FOOD & CONVENIENCE')]
df_7_eleven.loc[:,'Amount']=df_7_eleven['Amount']+2.0
df_7_eleven_coffee=df_7_eleven.copy()
df_7_eleven_coffee.loc[:,'Amount']=-2.00
df_7_eleven_coffee.loc[:,'Category']='COFFEE'
df_7_eleven_coffee.loc[:,'Master Category']='FOOD AND DINING'

df_7_eleven_transformed = pd.concat([df_7_eleven_coffee, df_7_eleven], ignore_index=True)


In [ ]:
df_final=pd.concat([filtered_df, df_7_eleven_transformed], ignore_index=True)

In [ ]:
len(df_final)
df_final.head(100)
len(df_final[(df_final['Transaction Date'] >= '2022-06-01') & (df_final['Transaction Date'] <= '2024-07-01')])
sum(df_final.isnull().any(axis=1))


In [ ]:
# filtered_df.groupby('Standardized Description')['Amount'].sum().sort_values(ascending=True)
# filtered_df['Amount'].sum()
df_final.to_csv('Transactions Full.csv')

